In [12]:
# Install dependencies as needed:

import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('3A2M.csv')

df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df = df.drop(columns=['Unnamed: 0'])

df.head()

,title,directions,NER,genre,label
0,Reeses Cups(Candy),"[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu...",drinks,2
1,Rhubarb Coffee Cake,"[""Cream sugar and butter."", ""Add egg and beat ...","[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou...",drinks,2
2,Quick Barbecue Wings,"[""Clean wings."", ""Flour and fry until done."", ...","[""chicken"", ""flour"", ""barbecue sauce""]",nonveg,3
3,Chocolate Frango Mints,"[""Mix ingredients together for 5 minutes."", ""S...","[""cake mix"", ""chocolate fudge pudding"", ""sour ...",drinks,2
4,Corral Barbecued Beef Steak Strips,"[""Brown strips in cooking oil."", ""Pour off dri...","[""long"", ""cooking oil"", ""tomato sauce"", ""water...",drinks,2
